In [ ]:
import os
import random
from DECIMER import predict_SMILES
from ase.data.pubchem import pubchem_atoms_search, pubchem_atoms_conformer_search
from rdkit import Chem
from rdkit.Chem import AllChem
from ase.io import read, write 
import matplotlib.pyplot as plt
from ase.visualize import view
import matplotlib.pyplot as plt
import numpy as np
from ase import io, Atoms
from ase.neighborlist import natural_cutoffs
from ase.build import sort
from ase.neighborlist import NeighborList
from ase.visualize import *
from ase.io import *
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate
from ase.build import bulk, surface
from ase.build.tools import sort
import random
from ase.build import molecule
from ase.neighborlist import NeighborList
import pandas as pd
from rdkit.Chem import rdmolops
from ase.build import molecule as build_molecule, add_adsorbate
from ase.constraints import FixAtoms
from ase.data import covalent_radii

def SmilesToMol(smiles):
    """
    Converts a SMILES string to an RDKit molecule, adds hydrogens,
    embeds the molecule in 3D space, and optimizes it.
    
    Args:
        smiles (str): SMILES representation of the molecule.
    
    Returns:
        mol: An RDKit Mol object if successful, otherwise None.
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Failed to create molecule from SMILES: {smiles}")
        return None
    
    # Add hydrogens to the molecule
    hmol = Chem.AddHs(mol)
    # Embed molecule in 3D space
    AllChem.EmbedMolecule(hmol)
    # Optimize the 3D structure
    AllChem.MMFFOptimizeMolecule(hmol)
    return hmol



def place_molecule_on_slab(slab: Atoms, molecule_name: str) -> Atoms:
    """
    Places a molecule on top of a given slab at a height determined by the top surface of the slab
    and the atomic radius of the molecule's first atom.

    Parameters:
        slab (Atoms): The slab structure on which the molecule will be placed.
        molecule_name (str): Name of the molecule to be placed (e.g., "H2O" for water).

    Returns:
        Atoms: Combined structure of the slab with the molecule on top.
    """
    
    # Load or create the molecule
    mol = molecule_name  # Create the molecule

    # Find the top of the slab by calculating the max z-coordinate
    max_z_slab = max(atom.position[2] for atom in slab)
    # Find the top of the slab by calculating the max z-coordinate
    min_z_slab = min(atom.position[2] for atom in slab)
    # Determine the atomic radius of the first atom of the molecule
    atomic_radius = covalent_radii[mol[0].number]  # Look up the atomic radius

    # Set the molecule just above the top surface of the slab
    slab_center_x = (slab.get_cell()[0, 0] + slab.get_cell()[1, 0]) / 2  # Center x-coordinate of slab
    slab_center_y = (slab.get_cell()[0, 1] + slab.get_cell()[1, 1]) / 2  # Center y-coordinate of slab
    add_adsorbate(slab, mol, height=-(max_z_slab-min_z_slab)-atomic_radius*4, position=(slab_center_x, slab_center_y))

    # Optionally, fix the bottom layer of the slab
#     min_z = min(atom.position[2] for atom in slab)  # Get minimum z-position of atoms in the slab
#     fixed_atoms = [atom.index for atom in slab if atom.position[2] < min_z + 0.1]  # Fix atoms near bottom
#     slab.set_constraint(FixAtoms(indices=fixed_atoms))

    # View the structure (optional)
    #view(slab)

    return slab


# Load the .xls file
file_path = './data/Mol_Group_A.xlsx'  # replace with the actual path to your .xls file
smiles_file = pd.read_excel(file_path)
SMILES= smiles_file['Smiles'].tolist()
molecules_name=smiles_file['Name'].tolist()

for i in range(len(SMILES)):
    smiles_fragment = SMILES[i]
    filename = molecules_name[i]
    organic_mol = SmilesToMol(smiles_fragment)

    if organic_mol is None:
        raise ValueError("No organic fragment was found.")
        
    output_mol_folder = 'fragment_mol_files'
    os.makedirs(output_mol_folder, exist_ok=True)
    output_mol_file = os.path.join(output_mol_folder, f'{filename}.mol')

    # Save the organic fragment containing nitrogen to a .mol file
    Chem.MolToMolFile(organic_mol, output_mol_file)

    # Read the .mol file with ASE
    molecule = read(output_mol_file)
    
    output_POSCAR_folder = 'slab_HTM'
    os.makedirs(output_POSCAR_folder, exist_ok=True)
    output_POSCAR_file = os.path.join(output_POSCAR_folder, f'{filename}_slab.vasp')
    print(output_POSCAR_file)

    surface = read('surface_001.vasp')
    combined_structure = place_molecule_on_slab(surface, molecule)
    write(output_POSCAR_file, combined_structure, format='vasp')

    # Plot the molecule
    fig, axes = plt.subplots(1, 1, figsize=(4, 4))
    plot_atoms(combined_structure, axes, radii=0.5, rotation=('90x,0y,00z'))
    plt.show()


#view(combined_structure, viewer='ngl')

In [ ]:
add_adsorbate?